In [2]:
import pandas as pd
import numpy as np

## Question 10: Rising Temperature


| column Name | Type |
|-------------|------|
|      id     |  int |
| recordDate  | date |
| temperature | int  |

Write an SQL query to find all dates `id` with higher temperature compared to its previous dates(yerterday)

Return the result table in **any order**.

The query result format is in the following example:


| id | recordDate | Temperature |
|----|------------|-------------|
| 1  | 2015-01-01 | 10          |
| 2  | 2015-01-02 | 25          |
| 3  | 2015-01-03 | 20          |
| 4  | 2015-01-04 | 30          |

**Please use python to solve this question**

In [2]:
Weather = pd.DataFrame({"id":[1,2,3,4],
             "recordDate":["2015-01-01","2015-01-02","2015-01-03","2015-01-04"],
             "Temperature":[10,25,20,30]})
Weather

,id,recordDate,Temperature
0,1,2015-01-01,10
1,2,2015-01-02,25
2,3,2015-01-03,20
3,4,2015-01-04,30


In [3]:
from datetime import datetime

wea_copy = Weather.copy()
wea_copy["date"] = wea_copy["recordDate"].apply(lambda date:datetime(year=int(date.split("-")[0]),
                                                      month=int(date.split("-")[1]),
                                                      day=int(date.split("-")[2])))
wea_copy

,id,recordDate,Temperature,date
0,1,2015-01-01,10,2015-01-01
1,2,2015-01-02,25,2015-01-02
2,3,2015-01-03,20,2015-01-03
3,4,2015-01-04,30,2015-01-04


In [4]:
sample_size = len(wea_copy["date"])
dates = wea_copy["date"]
temperatures = wea_copy["Temperature"]
IDs = []
for i in range(sample_size-1):
    if ((dates[i+1]-dates[i]).days==1) and (temperatures[i+1]>temperatures[i]):
        IDs.append(wea_copy["id"][i+1])      

In [42]:
pd.DataFrame({"id":IDs})

,id
0,2
1,4


## Question 2: Trips and Users

`Trips`

| Column Name | Type     |
|-------------|----------|
| Id          | int      |
| Client_Id   | int      |
| Driver_Id   | int      |
| City_Id     | int      |
| Status      | enum     |
| Request_at  | date     |     

Id is the primary key for this table.
The table holds all taxi trips. Each trip has a unique Id, while Client_Id and Driver_Id are foreign keys to the Users_Id at the Users table.
Status is an ENUM type of (‘completed’, ‘cancelled_by_driver’, ‘cancelled_by_client’).

`users`

| Column Name | Type     |
|-------------|----------|
| Users_Id    | int      |
| Banned      | enum     |
| Role        | enum     |

Users_Id is the primary key for this table.
The table holds all users. Each user has a unique Users_Id, and Role is an ENUM type of (‘client’, ‘driver’, ‘partner’).
Status is an ENUM type of (‘Yes’, ‘No’).

Write a SQL query to find the cancellation rate of requests with unbanned users (both client and driver must not be banned) each day between "2013-10-01" and "2013-10-03".

The cancellation rate is computed by dividing the number of canceled (by client or driver) requests with unbanned users by the total number of requests with unbanned users on that day.

Return the result table in any order. Round Cancellation Rate to two decimal points.

The query result format is in the following example:


In [59]:
Trips = pd.DataFrame({"Id":np.arange(1,11),
             "Client_Id":[1,2,3,4,1,2,3,2,3,4],
             "Driver_Id":[10,11,12,13,10,11,12,12,10,13],
             "City_Id":[1,1,6,6,1,6,6,12,12,12],
             "Status":["completed","Cancelled_by_driver","completed","cancelled_by_client"]+
                      ["completed"]*5+["cancelled_by_drive"],
             "Request_at":["2013-10-01"]*4+["2013-10-02"]*3+["2013-10-03"]*3})
Users = pd.DataFrame({"Users_Id":[1,2,3,4,10,11,12,13],
             "Banned":["No","Yes"]+["No"]*6,
             "ROle":["client"]*4+["Driver"]*4})

In [81]:
Trips

,Id,Client_Id,Driver_Id,City_Id,Status,Request_at
0,1,1,10,1,completed,2013-10-01
1,2,2,11,1,Cancelled_by_driver,2013-10-01
2,3,3,12,6,completed,2013-10-01
3,4,4,13,6,cancelled_by_client,2013-10-01
4,5,1,10,1,completed,2013-10-02
5,6,2,11,6,completed,2013-10-02
6,7,3,12,6,completed,2013-10-02
7,8,2,12,12,completed,2013-10-03
8,9,3,10,12,completed,2013-10-03
9,10,4,13,12,cancelled_by_drive,2013-10-03


In [101]:
# convert the str to datetime
trips = Trips.copy()
trips["Request_at"] = trips["Request_at"].apply(lambda date:datetime.strptime(date,"%Y-%m-%d"))

# filter out users whose status is banned
users = Users.copy()
users_no_banned = users[users["Banned"] != "Yes"]

# merge the two tables to ensure users in trips are all qualified users
merged_df = pd.merge(trips,users_no_banned,left_on="Client_Id",right_on="Users_Id",how='inner').merge(users_no_banned,
                                                                                          left_on="Driver_Id",
                                                                                          right_on="Users_Id")

# filter out dates that are wihtin the range of question
merged_df_ = merged_df[merged_df["Request_at"].isin(pd.date_range("2013-10-01","2013-10-03"))]

# compute the cannellation rate
cancellation = merged_df[["Client_Id","Status"]].groupby(merged_df["Request_at"])\
.apply(lambda x:sum(x["Status"]!="completed")/x.count())[["Client_Id"]]\
.rename(columns={"Client_Id":"Cancellation Rate"})
cancellation

,Cancellation Rate
Request_at,
2013-10-01,0.333333
2013-10-02,0.000000
2013-10-03,0.500000


In [102]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,0.343510,-0.548852
1,a,two,-0.509820,-1.878460
2,b,one,-0.766050,-1.374197
3,b,two,0.057398,0.685107
4,a,one,-0.048621,-0.366070


## Question 11: Game Play Analysis I

`Activity`


| Column Name  | Type    |
|--------------|---------|
| player_id    | int     |
| device_id    | int     |
| event_date   | date    |
| games_played | int     |

(player_id, event_date) is the primary key of this table.
This table shows the activity of players of some game.
Each row is a record of a player who logged in and played a number of games (possibly 0) before logging out on some day using some device.

Write an SQL query that reports the first login date for each player.

The query result format is in the following example:

Activity table:

| player_id | device_id | event_date | games_played |
|-----------|-----------|------------|--------------|
| 1         | 2         | 2016-03-01 | 5            |
| 1         | 2         | 2016-05-02 | 6            |
| 2         | 3         | 2017-06-25 | 1            |
| 3         | 1         | 2016-03-02 | 0            |
| 3         | 4         | 2018-07-03 | 5            |


In [153]:
Activity = pd.DataFrame({"player_Id":[1,1,2,3,3],
                        "devide_id":[2,2,3,1,4],
                        "event_date":pd.to_datetime(["2016-03-01","2016-05-02","2017-06-25","2016-03-02","2018-07-03"]),
                        "games_played":[5,6,1,0,5]})
Activity

,player_Id,devide_id,event_date,games_played
0,1,2,2016-03-01,5
1,1,2,2016-05-02,6
2,2,3,2017-06-25,1
3,3,1,2016-03-02,0
4,3,4,2018-07-03,5


In [144]:
# the first login date is the minimum event date
pd.DataFrame(Activity["event_date"].groupby(Activity["player_Id"]).min()).\
rename(columns={"event_date":"first_login"})

,first_login
player_Id,
1,2016-03-01
2,2017-06-25
3,2016-03-02


## Question 12: Game Play Analysis II

| Column Name  | Type    |
|--------------|---------|
| player_id    | int     |
| device_id    | int     |
| event_date   | date    |
| games_played | int     |

(player_id, event_date) is the primary key of this table.
This table shows the activity of players of some game.
Each row is a record of a player who logged in and played a number of games (possibly 0) before logging out on some day using some device.

Write a SQL query that reports the device that is first logged in for each player.

The query result format is in the following example:

`Activity`

| player_id | device_id | event_date | games_played |
|-----------|-----------|------------|--------------|
| 1         | 2         | 2016-03-01 | 5            |
| 1         | 2         | 2016-05-02 | 6            |
| 2         | 3         | 2017-06-25 | 1            |
| 3         | 1         | 2016-03-02 | 0            |
| 3         | 4         | 2018-07-03 | 5            |


**Please use python to solve this question**

In [147]:
# first we find out the first login date for each player
login_df = Activity["event_date"].groupby(Activity["player_Id"]).min().to_frame().reset_index()

# then merge login_df with original dataframe to find the device corresponding to the min date
pd.merge(Activity,login_df,on=["player_Id","event_date"]).loc[:,["player_Id","devide_id"]]

,player_Id,devide_id
0,1,2
1,2,3
2,3,1


## Question 13: Game Play Analysis III

`Activity`

| Column Name  | Type    |
|--------------|---------|
| player_id    | int     |
| device_id    | int     |
| event_date   | date    |
| games_played | int     |

(player_id, event_date) is the primary key of this table.
This table shows the activity of players of some game.
Each row is a record of a player who logged in and played a number of games (possibly 0) before logging out on some day using some device.

Write an SQL query that reports for each player and date, how many games played so far by the player. That is, the total number of games played by the player until that date. Check the example for clarity.

The query result format is in the following example:

**Please use python to solve this question**

In [151]:
Activity_ = pd.DataFrame({"player_Id":[1,1,1,3,3],
                        "devide_id":[2,2,3,1,4],
                        "event_date":pd.to_datetime(["2016-03-01","2016-05-02","2017-06-25","2016-03-02","2018-07-03"]),
                        "games_played":[5,6,1,0,5]})

In [152]:
sorted_act_df = Activity_.sort_values(by=["player_Id","event_date"],ascending=[True,True])
sorted_act_df["games_played_so_far"] = sorted_act_df.groupby("player_Id").agg({"games_played":"cumsum"})
sorted_act_df[["player_Id","event_date","games_p
               layed_so_far"]].set_index("player_Id")

,event_date,games_played_so_far
player_Id,,
1,2016-03-01,5
1,2016-05-02,11
1,2017-06-25,12
3,2016-03-02,0
3,2018-07-03,5


## Question 14: 

Write an SQL query that reports the fraction of players that logged in again on the day after the day they first logged in, rounded to 2 decimal places. In other words, you need to count the number of players that logged in for at least two consecutive days starting from their first login date, then divide that number by the total number of players.

**Please use python to solve this question**

In [200]:
Activity = pd.DataFrame({"player_Id":[1,1,2,3,3],
                        "devide_id":[2,2,3,1,4],
                        "event_date":pd.to_datetime(["2016-03-01","2016-03-02","2017-06-25","2016-03-02","2018-07-03"]),
                        "games_played":[5,6,1,0,5]})

In [210]:
# this is a typical question about computation of next-day retention
# first we find out the login date

from pandas.tseries.offsets import Day
act = Activity.copy()
act["first_login"] = Activity["event_date"].groupby(Activity["player_Id"]).transform(lambda x:x.min()).values

oneday = Day(1)
act['isConsecutive'] = (act["event_date"] - act["first_login"] == oneday).values
act_ = act[act['event_date'].isin(act["first_login"].unique())]
first_login_num = len(act_["player_Id"].unique())
retention = sum(act_["isConsecutive"])
pd.DataFrame({"fraction":[round(retention/first_login_num,2)]})

,fraction
0,0.33


## Question 15

The Employee table holds all employees. The employee table has three columns: Employee Id, Company Name, and Salary.

|Id   | Company    | Salary |
|-----|------------|--------|
|1    | A          | 2341   |
|2    | A          | 341    |
|3    | A          | 15     |
|4    | A          | 15314  |
|5    | A          | 451    |
|6    | A          | 513    |
|7    | B          | 15     |
|8    | B          | 13     |
|9    | B          | 1154   |
|10   | B          | 1345   |
|11   | B          | 1221   |
|12   | B          | 234    |
|13   | C          | 2345   |
|14   | C          | 2645   |
|15   | C          | 2645   |
|16   | C          | 2652   |
|17   | C          | 65     |

Write a SQL query to find the median salary of each company. Bonus points if you can solve it without using any built-in SQL functions.

*The result should be same as the dataframe below*

|Id   | Company    | Salary |
|-----|------------|--------|
|5    | A          | 451    |
|6    | A          | 513    |
|12   | B          | 234    |
|9    | B          | 1154   |
|14   | C          | 2645   |



**Please use python to solve this question. Note: you cannot use median function**

In [3]:
Employee = pd.DataFrame({"Id":list(range(1,18)),
                        "Company":["A"]*6+["B"]*6+["C"]*5,
                        "Salary":[2341,341,15,15134,451,513,15,13,1154,1345,1221,234,2345,2645,2645,2652,65]})

In [257]:
# add one clumn to indicate the number of employees for each company
employee = Employee.copy()
employee["emp_cnt"] = employee.groupby("Company").transform(lambda x:x.count()).iloc[:,0].values

# add another column to indicate the row number for each employee within each company
sorted_emp = employee.sort_values(["Company","Salary"],ascending=[True,True])
sorted_emp["row_number"] = sorted_emp["Salary"].groupby(sorted_emp["Company"]).rank(ascending=True,method="first")

# For both odd and even number observations, the median number should be wihtin the range 
# (number_of_obervations/2,number_of_obervations/2+1,number_of_obervations/2+0.5)
mask = sorted_emp.groupby("Company").\
apply(lambda group:group["row_number"].\
      isin([group["emp_cnt"].unique()[0]/2,group["emp_cnt"].unique()[0]/2+1,group["emp_cnt"].unique()[0]/2+0.5]))
sorted_emp[mask.values].loc[:,["Id","Company","Salary"]]

,Id,Company,Salary
4,5,A,451
5,6,A,513
11,12,B,234
8,9,B,1154
13,14,C,2645


## Question 16: Managers with at least 5 Direct Reports

The `Employee` table holds all employees including their managers. Every employee has an Id, and there is also a column for the manager Id.

|Id    |Name 	  |Department |ManagerId |
|------|----------|-----------|----------|
|101   |John 	  |A 	      |null      |
|102   |Dan 	  |A 	      |101       |
|103   |James 	  |A 	      |101       |
|104   |Amy 	  |A 	      |101       |
|105   |Anne 	  |A 	      |101       |
|106   |Ron 	  |B 	      |101       |

Given the Employee table, write a SQL query that finds out managers with at least 5 direct report. For the above table, your SQL query should return:

In [6]:
Employee = pd.DataFrame({"Id":list(range(101,107,)),
             "Name":["John","Dan","James","Amy","Anne","Ron"],
             "Department":["A"]*5+["B"],
             "ManagerId":[np.nan]+[101]*5})
Employee

,Id,Name,Department,ManagerId
0,101,John,A,NaN
1,102,Dan,A,101.0
2,103,James,A,101.0
3,104,Amy,A,101.0
4,105,Anne,A,101.0
5,106,Ron,B,101.0


In [25]:
merged_df = pd.merge(Employee,Employee,left_on="ManagerId",right_on="Id",suffixes=("_employee","_manager"))
manager = merged_df.groupby("Name_manager").agg({"Name_employee":'count'}).reset_index()[["Name_manager"]].\
rename(columns={"Name_manager":"Employee"})
manager

,Employee
0,John


## Question 17: Find Median Given Frequency of Numbers

The `Numbers` table keeps the value of number and its frequency.

|  Number  |  Frequency  |
|----------|-------------|
|  0       |  7          |
|  1       |  1          |
|  2       |  3          |
|  3       |  1          |

In this table, the numbers are 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 3, so the median is (0 + 0) / 2 = 0.

In [26]:
Numbers = pd.DataFrame({"Number":[0,1,2,3],
             "Frequency":[7,1,3,1]})
Numbers

,Number,Frequency
0,0,7
1,1,1
2,2,3
3,3,1


In [69]:
# sort the values based on an ascending order
numbers = Numbers.copy()
numbers["asc_cumsum"]  = numbers["Frequency"].cumsum()

# sort the values based on a descening order

desc_cumsum = numbers.sort_values(by="Number",ascending=False).cumsum()["Frequency"]
numbers["desc_cumsum"] = desc_cumsum
# filter out the median number
mask = (numbers["asc_cumsum"]>=numbers["Frequency"].sum()/2) & (numbers["desc_cumsum"] >= numbers["Frequency"].sum()/2)
median = numbers[mask][["Number"]]
median

,Number
0,0


## Question 18: Winning Candidate

`candidate`

| id  | Name    |
|-----|---------|
| 1   | A       |
| 2   | B       |
| 3   | C       |
| 4   | D       |
| 5   | E       |

`vote`

| id  | CandidateId  |
|-----|--------------|
| 1   |     2        |
| 2   |     4        |
| 3   |     3        |
| 4   |     2        |
| 5   |     5        |

id is the auto-increment primary key,
CandidateId is the id appeared in Candidate table.

Write a sql to find the name of the winning candidate, the above example will return the winner B.

In [71]:
candidate = pd.DataFrame({"id":[1,2,3,4,5],
             "Name":["A","B","C","D","E"]})
vote = pd.DataFrame({"id":[1,2,3,4,5],
                    "CandidateId":[2,4,3,2,5]})

In [79]:
merged_df

,id,Name,CandidateId
0,1,A,2
1,2,B,4
2,3,C,3
3,4,D,2
4,5,E,5


In [104]:
merged_df = pd.merge(candidate,vote,left_on="id",right_on="CandidateId")
votes = merged_df["CandidateId"].groupby(merged_df["CandidateId"]).count()
mask = merged_df["CandidateId"] == votes.sort_values(ascending=False).index[0]
winner = pd.DataFrame(merged_df[mask]["Name"].unique()).rename(columns={0:"Name"})
winner

,Name
0,B


## Question 19: Get Highest Answer Rate Question

Get the highest answer rate question from a table survey_log with these columns: id, action, question_id, answer_id, q_num, timestamp.

id means user id; action has these kind of values: "show", "answer", "skip"; answer_id is not null when action column is "answer", while is null for "show" and "skip"; q_num is the numeral order of the question in current session.

Write a sql query to identify the question which has the highest answer rate.

| id   | action    | question_id  | answer_id  | q_num     | timestamp  |
|------|-----------|--------------|------------|-----------|------------|
| 5    | show      | 285          | null       | 1         | 123        |
| 5    | answer    | 285          | 124124     | 1         | 124        |
| 5    | show      | 369          | null       | 2         | 125        |
| 5    | skip      | 369          | null       | 2         | 126        |



In [107]:
survey_log = pd.DataFrame({"id":[5]*4,
                            "action":["show","answer","show","skip"],
                            "question_id":[285,285,369,369],
                            "answer_id":[np.nan,124124,np.nan,np.nan],
                            "q_num":[1,1,2,2],
                            "timestamp":[123,124,125,126]})
survey_log

,id,action,question_id,answer_id,q_num,timestamp
0,5,show,285,NaN,1,123
1,5,answer,285,124124.0,1,124
2,5,show,369,NaN,2,125
3,5,skip,369,NaN,2,126


**Analysis**

The number of quesiton is always one but the frequency of being answered is different from question to question. So we just need to compute the frequency of being answered so as to get the question that gets the highest answer rate.

In [134]:
answer_ = survey_log[["question_id","answer_id"]].groupby(survey_log["question_id"]).agg({"answer_id":"count"}).\
rename(columns={"answer_id":"frequency"}).reset_index()
pd.DataFrame({"survey_log":[answer_["question_id"][0]]})

,survey_log
0,285


## Question 20: Find Cumulative Salary of an Employee

Write an SQL query to calculate the **cumulative salary summary** for every employee in a single unified table.

The cumulative salary summary for an employee can be calculated as follows:

- For each month that the employee worked, sum up the salaries in that month and the previous two months. This is their 3-month sum for that month. If an employee did not work for the company in previous months, their effective salary for those months is 0.
- Do not include the 3-month sum for the most recent month that the employee worked for in the summary.
- Do not include the 3-month sum for any month the employee did not work.

Return the result table ordered by Id in ascending order. In case of a tie, order it by Month in descending order.

| Id | Month | Salary |
|----|-------|--------|
| 1  | 1     | 20     |
| 2  | 1     | 20     |
| 1  | 2     | 30     |
| 2  | 2     | 30     |
| 3  | 2     | 40     |
| 1  | 3     | 40     |
| 3  | 3     | 60     |
| 1  | 4     | 60     |
| 3  | 4     | 70     |
| 1  | 7     | 90     |
| 1  | 8     | 90     |

`result table`

| id | month | Salary |
|----|-------|--------|
| 1  | 4     | 130    |
| 1  | 3     | 90     |
| 1  | 2     | 50     |
| 1  | 1     | 20     |
| 2  | 1     | 20     |
| 3  | 3     | 100    |
| 3  | 2     | 40     |



In [39]:
# first we find out the most recent month for each employee
Employee = pd.DataFrame({"Id":[1,2,1,2,3,1,3,1,3,1,1],
             "Month":[1,1,2,2,2,3,3,4,4,7,8],
             "Salary":[20,20,30,30,40,40,60,60,70,90,90]})
Employee

,Id,Month,Salary
0,1,1,20
1,2,1,20
2,1,2,30
3,2,2,30
4,3,2,40
5,1,3,40
6,3,3,60
7,1,4,60
8,3,4,70
9,1,7,90


In [40]:
# drop the observations with the most recent moonth from the table
employee = Employee.copy()
max_month = Employee["Month"].groupby(Employee["Id"]).transform("max")
No_Maxmonth_Emp = employee[employee["Month"] != max_month]
nomax_emp = No_Maxmonth_Emp.sort_values(["Id","Month"],ascending=[True,True]) 

In [41]:
merged_df = pd.merge(nomax_emp,nomax_emp,how="cross")
trans_data = merged_df[(((merged_df["Month_x"] - merged_df["Month_y"])==0)&(merged_df["Id_x"] == merged_df["Id_y"]))|
                       (((merged_df["Month_x"] - merged_df["Month_y"])==1)&(merged_df["Id_x"] == merged_df["Id_y"]))|
                       (((merged_df["Month_x"] - merged_df["Month_y"])==2)&(merged_df["Id_x"] == merged_df["Id_y"]))]

res = trans_data.groupby(["Id_x","Month_x"]).agg({"Salary_y":"sum"}).reset_index()
res

,Id_x,Month_x,Salary_y
0,1,1,20
1,1,2,50
2,1,3,90
3,1,4,130
4,1,7,90
5,2,1,20
6,3,2,40
7,3,3,100
